In [33]:
# Snowpark Libs
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import col,to_date, udf, concat_ws, call_function, lit, concat
# Data Science Libs
import pandas as pd
import os
import dotenv
from pathlib import Path
from datetime import datetime
import numpy as np
import datetime
import pandas as pd
import re
#Snowflake Cortext
from snowflake.cortex import Complete, ExtractAnswer, Sentiment, Summarize, Translate, ClassifyText
# If there are any certificates issues, run the command: /Applications/Python\ 3.9/Install\ Certificates.command

In [2]:
# Read config parser .ini file with your connection information

Path = ".env"

# SSO - KochID
connection_parms = {
    "account": os.getenv('account'),
    "user": os.getenv('email'),
    "authenticator": os.getenv('authenticator'),
    "database": os.getenv('database'),
    "schema": os.getenv('schema'),
    "warehouse": os.getenv('warehouse'),"role": os.getenv('role')
} 

connection_parms

# Create Snowflake Session object
session = Session.builder.configs(connection_parms).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://auth.kochid.com/idp/SSO.saml2?SAMLRequest=pZJfb9owFMW%2FSuQ9x3YSUDuLUDFQOzQ2UBM2qW8mNuDh2JmvQ8q3n8MfqXtoX%2FZm2ef6d%2B49d%2FTwWuvoKB0oa3KUYIoiaSorlNnlaF0%2BxvcoAs%2BN4NoamaOTBPQwHgGvdcMmrd%2BbZ%2FmnleCj8JEB1j%2FkqHWGWQ4KmOG1BOYrVky%2BL1iKKeMA0vmAQ9cSASqw9t43jJCu63CXYet2JKWUEvqZBFUv%2BYTeIJqPGY2z3lZW30peQ0%2FvIBJCBz0iKAJhdS38osxlBB9RNhcRsK9luYpXy6JE0eTW3dQaaGvpCumOqpLr58XFAAQHyhxVP1RrglGBW4glBx%2BnmHeAwdhuq%2FlBVrZuWh8AOJzIVgqi7U6Fsc1nOWoOSlgth%2FNiM1S7xebXMTu9bCdPT4dk8Pub64q0Wu%2BXtGumm%2BzO0ApFP28hp33Ic4BWzk3vwocrmg5jmsVZUiYDllKW3uNkcPeColmIVhnuz5U3%2Fzzkjg%2B22itx9qZEQ4piiftkUnTZDnYmuPF%2F9Dwibz%2B67tyPEMN8trJaVafo0bqa%2B%2FdTSnByvlEi3p6lTNZc6YkQTgKEtLS23dRJ7sNqe9dKRMYX6r%2FLPf4L&RelayState=ver%3A1-hint%3A977594690174982-ETMsDgAAAZ

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.


In [3]:
connection_parms

{'account': 'invistanonprod.us-east-2.aws',
 'user': 'chitrakumarsai.chenurivenkata@invista.com',
 'authenticator': 'externalbrowser',
 'database': 'DS_D_WORKSPACE',
 'schema': 'SANDBOX',
 'warehouse': 'DS_D_DEFAULT_WH',
 'role': 'DS_D_SCIENTIST_FR'}

In [8]:
nylon_recycling = pd.read_excel(r"./data/raw_data/N66 Recycle Depoly Field_010925 Test_2025-03-30.xlsx", skiprows=1)
nylon_recycling.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                         col.replace(' ', "_")).upper() for col in nylon_recycling.columns]
session.create_dataframe(nylon_recycling).write.mode("overwrite").save_as_table("NYLON_RECYCLING_SAMPLE")

In [9]:
session.table("NYLON_RECYCLING_SAMPLE").show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PUBLICATION_NUMBER"  |"ASSIGNEE_APPLICANT"                                |"PRIORITY_DATE"                       |"PUBLICATION_DATE"  |"TITLE__ENGLISH_"                                   |"ABSTRACT__ENGLISH_"                                |"CLAIMS__ENGLISH_"                                  |"TITLE___DWPI"                                      |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [42]:
# Read the table and limit to the first two rows
df = session.table("NYLON_RECYCLING_SAMPLE").limit(2)

# Combine the title, abstract, and claims columns into a single text column
df_combined = df.with_column(
    "COMBINED_TEXT",
    concat(
        lit("TITLE: "), col("TITLE__ENGLISH_"), lit("\nABSTRACT: "), col("ABSTRACT__ENGLISH_"),
        lit("\nCLAIMS: "), col("CLAIMS__ENGLISH_")
    )
)

In [43]:
df_combined.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PUBLICATION_NUMBER"  |"ASSIGNEE_APPLICANT"              |"PRIORITY_DATE"          |"PUBLICATION_DATE"  |"TITLE__ENGLISH_"                                   |"ABSTRACT__ENGLISH_"                                |"CLAIMS__ENGLISH_"                                  |"TITLE___DWPI"                                      |"COMBINED_TEXT"                                     |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [44]:

df_combined = df_combined.withColumn(
    "SUMMARY",
    Summarize(col("COMBINED_TEXT"))
)
df_combined.collect()

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_87951/3111339128.py:3: DeprecationWarning: Summarize() is deprecated and will be removed in a future release. Use summarize() instead
  Summarize(col("COMBINED_TEXT"))


[Row(PUBLICATION_NUMBER='US12252606B2', ASSIGNEE_APPLICANT='Evonik Operations GmbH,Essen,DE', PRIORITY_DATE='2018-09-21 | 2019-09-19', PUBLICATION_DATE='2025-03-18', TITLE__ENGLISH_='Composites having a thermoplastic matrix', ABSTRACT__ENGLISH_='\n\nThe present invention is directed to continuous fiber-reinforced composites having a matrix of modified PVDF. The composite may be a semifinished product or a product or finished part manufactured therefrom. A product according to the invention is pelletized long-fiber material, while a finished part according to the invention is especially a thermoplastic composite pipe.', CLAIMS__ENGLISH_='\nThe invention claimed is: \n | 1. A composite comprising: \na polymer matrix and \ncontinuous fibers embedded therein, \nwherein the polymer matrix has a content of at least 80% by weight of a mixture comprising the following components: \n(a) low-viscosity polyvinylidene fluoride (PVDF) as the main component, present in a content of at least 50 parts

In [45]:
df_combined.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PUBLICATION_NUMBER"  |"ASSIGNEE_APPLICANT"              |"PRIORITY_DATE"          |"PUBLICATION_DATE"  |"TITLE__ENGLISH_"                                   |"ABSTRACT__ENGLISH_"                                |"CLAIMS__ENGLISH_"                                  |"TITLE___DWPI"                                      |"COMBINED_TEXT"                                     |"SUMMARY"                                           |
------------------------------------------------------------------------------------------------------------------------------------------------------

In [52]:
df_summary = df_combined.to_pandas()

In [53]:
session.create_dataframe(df_summary).write.mode("overwrite").save_as_table("NYLON_RECYCLING_SUMMARY")

In [54]:
df_summary.to_excel(r"./data/processed/Nylon_Recycling_Summary.xlsx", index=False)

### Complete

In [62]:
# Read the table and limit to the first two rows
df = session.table("NYLON_RECYCLING_SAMPLE").limit(2)

# Combine the title, abstract, and claims columns into a single text column
df_combined = df.with_column(
    "COMBINED_TEXT",
    concat(
        lit("TITLE: "), col("TITLE__ENGLISH_"), lit("\nABSTRACT: "), col("ABSTRACT__ENGLISH_"),
        lit("\nCLAIMS: "), col("CLAIMS__ENGLISH_")
    )
)
# Define your instructions and context for the summarization
instructions = "Summarize the following patent information concisely in no more than 300 words, keeping all key points."
context = "The patent information includes title, abstract, and claims."

# Now call the COMPLETE function using the provided pattern:
# Complete(model, f"Instructions:{instructions}, Context:{context}, Prompt:{prompt}")
df_summary = df_combined.with_column(
    "SUMMARY",
    call_function(
        "SNOWFLAKE.CORTEX.COMPLETE",
        lit("snowflake-arctic"),  # Change to your preferred model if needed
        concat(
            lit(f"Instructions: {instructions}, Context: {context}, Prompt: "),
            col("COMBINED_TEXT")
        )
    )
)

In [ ]:
# Select the output columns and show the results
df_result = df_summary.select("PUBLICATION_NUMBER", "SUMMARY", "TITLE__ENGLISH_", "ABSTRACT__ENGLISH_", "CLAIMS__ENGLISH_")
df_result.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PUBLICATION_NUMBER"  |"SUMMARY"                                           |"TITLE__ENGLISH_"                                   |"ABSTRACT__ENGLISH_"                                |"CLAIMS__ENGLISH_"                                  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|US12252606B2          | The patent information is about a composite ma...  |Composites having a thermoplastic matrix            |                                                    |                                                    |
|                      |                            

In [64]:
# Read the table and limit to the first two rows
df = session.table("NYLON_RECYCLING_SAMPLE").limit(2)

# Combine the title, abstract, and claims columns into a single text column
df_combined = df.with_column(
    "COMBINED_TEXT",
    concat(
        lit("TITLE: "), col("TITLE__ENGLISH_"), lit("\nABSTRACT: "), col("ABSTRACT__ENGLISH_"),
        lit("\nCLAIMS: "), col("CLAIMS__ENGLISH_")
    )
)
# Define your instructions and context for the summarization
instructions = "Summarize the following patent information concisely in no more than 300 words, keeping all key points."
messages = [
    {"role": "system", "content": "You are an AI that summarizes text concisely. Do not introduce new information or assumptions."},
    {"role": "user", "content": "Summarize the following combined patent information while keeping all key points:\n{text}"}
]
# Creating the context string in the desired format
context = ",".join(f"role:{message['role']} content:{message['content']}" for message in messages)
# Now call the COMPLETE function using the provided pattern:
# Complete(model, f"Instructions:{instructions}, Context:{context}, Prompt:{prompt}")
df_summary = df_combined.with_column(
    "SUMMARY_2",
    call_function(
        "SNOWFLAKE.CORTEX.COMPLETE",
        lit("snowflake-arctic"),  # Change to your preferred model if needed
        concat(
            lit(f"Instructions: {instructions}, Context: {context}, Prompt: "),
            col("COMBINED_TEXT")
        )
    )
)

In [66]:
# # Select the output columns and show the results
# df_result = df_summary.select("PUBLICATION_NUMBER", "SUMMARY_2", "TITLE__ENGLISH_", "ABSTRACT__ENGLISH_", "CLAIMS__ENGLISH_")
# df_result.show()

In [ ]:
from snowflake.snowpark import functions as F
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, concat, lit, when
from snowflake.cortex import complete

# Define your prompt for summarizing the patent information
prompt = """[INST]
### 
You are an AI that summarizes patent information concisely. 
The patent information includes title, abstract, and claims. 
Summarize this information into a concise summary while keeping all key points.
Do not introduce new information or assumptions. 
### 
Prompt: """

# Assuming df_combined already has the combined text from title, abstract, and claims
df_combined = df.with_column(
    "COMBINED_TEXT",
    concat(
        lit("TITLE: "), col("TITLE__ENGLISH_"),
        lit("\nABSTRACT: "), col("ABSTRACT__ENGLISH_"),
        lit("\nCLAIMS: "), col("CLAIMS__ENGLISH_")
    )
)

# Use the cortex.Complete function directly to generate the summary
df_summary = df_combined.with_column(
    "SUMMARY",
    complete(
        'snowflake-arctic',  # Use your preferred model (e.g., "snowflake-arctic")
        F.concat(F.lit(prompt), F.col("COMBINED_TEXT"), F.lit("[/INST]"))
    )
)

# Clean the summary if necessary (adjustments for the actual output format)
df_result = df_summary.select("PUBLICATION_NUMBER", "SUMMARY", "TITLE__ENGLISH_", "ABSTRACT__ENGLISH_", "CLAIMS__ENGLISH_")

# Display the result (or save it to a file as needed)
df_result.show()


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PUBLICATION_NUMBER"  |"SUMMARY"                                           |"TITLE__ENGLISH_"                                   |"ABSTRACT__ENGLISH_"                                |"CLAIMS__ENGLISH_"                                  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|US12252606B2          | The patent is about a composite material made ...  |Composites having a thermoplastic matrix            |                                                    |                                                    |
|                      |                            